## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-18-16-56-06 +0000,nyt,Live Updates: European Leaders Arrive at White...,https://www.nytimes.com/live/2025/08/18/us/tru...
1,2025-08-18-16-55-03 +0000,startribune,Zelenskyy brings Europe's top leaders with him...,https://www.startribune.com/zelenskyy-brings-e...
2,2025-08-18-16-45-59 +0000,nypost,"OpenAI CEO Sam Altman warns of AI bubble, says...",https://nypost.com/2025/08/18/business/openai-...
3,2025-08-18-16-44-10 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/08/18/us/tru...
4,2025-08-18-16-44-00 +0000,wsj,"President Trump continues his on-again, off-ag...",https://www.wsj.com/politics/policy/trump-elec...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,44
8,zelensky,22
126,ukraine,15
71,will,12
331,canada,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
154,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...,135
106,2025-08-18-10-27-12 +0000,nypost,Harrowing photo shows teen killed in Russian s...,https://nypost.com/2025/08/18/world-news/russi...,121
96,2025-08-18-11-37-54 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...,114
93,2025-08-18-12-03-49 +0000,wapo,"What to expect as Zelensky, European leaders m...",https://www.washingtonpost.com/world/2025/08/1...,108
108,2025-08-18-10-10-29 +0000,nyt,Zelensky Calls for ‘Lasting’ Peace in Ukraine ...,https://www.nytimes.com/2025/08/18/us/politics...,103


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
154,135,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...
34,54,2025-08-18-15-31-36 +0000,bbc,'No-one was expecting this': European leaders ...,https://www.bbc.com/news/articles/c9d0v8ljw3no...
50,43,2025-08-18-15-00-03 +0000,nyt,Air Canada Flight Attendant Strike Declared Il...,https://www.nytimes.com/2025/08/18/world/canad...
19,37,2025-08-18-16-07-43 +0000,nypost,Israeli hostages will only be freed ‘when Hama...,https://nypost.com/2025/08/18/world-news/israe...
65,34,2025-08-18-14-15-38 +0000,nypost,Illegal migrant suspect in deadly Florida cras...,https://nypost.com/2025/08/18/us-news/illegal-...
88,34,2025-08-18-12-31-57 +0000,nypost,Texas ICE shooting suspects tied to secretive ...,https://nypost.com/2025/08/18/us-news/texas-ic...
7,32,2025-08-18-16-32-35 +0000,nypost,Golfer struck by lightning on New Jersey cours...,https://nypost.com/2025/08/18/us-news/golfer-s...
4,26,2025-08-18-16-44-00 +0000,wsj,"President Trump continues his on-again, off-ag...",https://www.wsj.com/politics/policy/trump-elec...
43,26,2025-08-18-15-18-49 +0000,nypost,Marvel leaves Georgia to film superhero movies...,https://nypost.com/2025/08/18/business/marvel-...
100,25,2025-08-18-11-00-00 +0000,wsj,"Rob Sand, a Democratic contender for Iowa’s go...",https://www.wsj.com/politics/meet-the-democrat...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
